## Trabalho WIN
Notebook base para o trabalho individual de mini-índice

In [1]:
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for ta: started
  Running setup.py install for ta: finished with status 'done'


  DEPRECATION: ta is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [2]:
import pandas as pd
import numpy as np
from ta.utils import dropna
from ta.volatility import BollingerBands
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('data.csv', sep='\t')
data

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,TICKVOL,VOL,SPREAD
0,2019.12.09,09:00:00,110930,111000,110910,110990,3447,11586,5
1,2019.12.09,09:01:00,110990,111125,110975,111075,9410,36048,5
2,2019.12.09,09:02:00,111080,111185,111080,111180,5629,22859,5
3,2019.12.09,09:03:00,111180,111185,111125,111175,3795,15209,5
4,2019.12.09,09:04:00,111180,111215,111165,111205,3308,11132,5
...,...,...,...,...,...,...,...,...,...
96932,2020.08.31,17:50:00,99880,99910,99745,99745,5077,18100,5
96933,2020.08.31,17:51:00,99750,99775,99700,99715,2119,7177,5
96934,2020.08.31,17:52:00,99715,99765,99715,99750,1133,3541,5
96935,2020.08.31,17:53:00,99745,99765,99680,99695,1232,4309,5


In [3]:
# Tratamentos básicos de datas
data['datetime'] = data['DATE'] + ' ' + data['TIME']
data['datetime'] = pd.to_datetime(data['datetime'])
data.sort_values(by = 'datetime', inplace = True)
data = data[['datetime', 'TICKVOL', 'CLOSE']]

# Renomeando coluna
data.rename(columns = {'CLOSE':'valor'}, inplace = True)
data.loc[:,'target_num'] = data['valor'].shift(-1)
data.loc[:,'target_diff'] = data['target_num']  - data['valor']
data.loc[:,'target'] = data['target_diff'].apply(lambda x: 1 if x>=10 else 0)
data = data.drop(columns = ['target_num', 'target_diff'])

data.head()


D:\Softwares\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
D:\Softwares\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,datetime,TICKVOL,valor,target
0,2019-12-09 09:00:00,3447,110990,1
1,2019-12-09 09:01:00,9410,111075,1
2,2019-12-09 09:02:00,5629,111180,0
3,2019-12-09 09:03:00,3795,111175,1
4,2019-12-09 09:04:00,3308,111205,0


In [4]:
# remover primeira linha do dia. Outra opção seria deixar estas linhas e manter a feature primeira_do_dia, para informar que aquele ponto pode estar sujeito a mais oscilações. Esta estratégia não é ideal no caso do uso de modelos lineares
data['primeira_do_dia'] = data['datetime'].diff() > pd.Timedelta(10, unit = 'hour')
data = data.loc[data['primeira_do_dia'] == False]
data = data[['datetime', 'valor', 'TICKVOL','target']].reset_index(drop = True)
data.head()

,datetime,valor,TICKVOL,target
0,2019-12-09 09:00:00,110990,3447,1
1,2019-12-09 09:01:00,111075,9410,1
2,2019-12-09 09:02:00,111180,5629,0
3,2019-12-09 09:03:00,111175,3795,1
4,2019-12-09 09:04:00,111205,3308,0


In [5]:
# Ex dos dados de um dia
data.loc[(data.datetime >= '2020-08-03') & (data.datetime <= '2020-08-04')]

,datetime,valor,TICKVOL,target
85557,2020-08-03 09:01:00,103985,10386,0
85558,2020-08-03 09:02:00,103935,6886,1
85559,2020-08-03 09:03:00,104015,5636,1
85560,2020-08-03 09:04:00,104050,6373,0
85561,2020-08-03 09:05:00,104045,4192,0
...,...,...,...,...
86086,2020-08-03 17:50:00,103045,3508,1
86087,2020-08-03 17:51:00,103055,1102,1
86088,2020-08-03 17:52:00,103070,572,1
86089,2020-08-03 17:53:00,103090,1201,1


In [6]:
# Definindo funções básicas
def acuracia(y_real, y_pred):
    return sum(y_real == y_pred) / len(y_pred)

# Separação de treino e teste. Note que, caso seja necessário um teste de validação, este será extraído do train
def split_treino_test(data):
    train = data[data['datetime'] < '2020-08-01']
    teste = data[data['datetime'] >= '2020-08-01']
    return train, teste

### Criação de um modelo baseline

Frequentemente é uma boa prática criar um modelo baseline utilizando alguma estratégia 'naive', para servir como comparação para os modelos que serão criados. Para este caso, consideraremos que o próximo minuto será

In [7]:
# Criação de variável target com lag

_, teste = split_treino_test(data)

teste_naive = teste.copy()

teste_naive['target_l1'] = teste_naive['target'].shift(1)
teste_naive=teste_naive.dropna()

# A predição é simplesmente o time step anterior
performance_baseline = acuracia(teste_naive['target'], teste_naive['target_l1'])
print("A acurácia do modelo Naive é " + str(round(100*performance_baseline,1)) + "%")

A acurácia do modelo Naive é 51.9%


In [8]:
# Outra predição Naive seria assumir que todos os resultados são 0:
qtdade_zeros_base = 1-teste_naive['target'].mean()
print("A acurácia assumindo que todos são 0 é " + str(round(100*qtdade_zeros_base,1)) + "%")

A acurácia assumindo que todos são 0 é 56.3%


### Exemplo de análise técnica como feature

In [9]:
# Initialize Bollinger Bands Indicator
data_bb = data.copy()
indicator_bb = BollingerBands(close=data_bb["valor"])
data_bb['bb_bbp'] = indicator_bb.bollinger_pband()
data_bb = data_bb.dropna()

data_bb.head()

,datetime,valor,TICKVOL,target,bb_bbp
19,2019-12-09 09:19:00,111255,2992,0,0.734259
20,2019-12-09 09:20:00,111255,1895,0,0.765122
21,2019-12-09 09:21:00,111225,1945,0,0.569301
22,2019-12-09 09:22:00,111215,1931,1,0.472465
23,2019-12-09 09:23:00,111230,1544,0,0.580744


### Exemplo do uso do Keras para regressão logística

In [10]:
from tensorflow import keras

# criando variaveis lagadas
data_bb['valor_l1'] = data_bb['valor'].shift(1)
data_bb['valor_l2'] = data_bb['valor'].shift(2)
data_bb['valor_l3'] = data_bb['valor'].shift(3)

data_bb['diff_l1'] = data_bb['valor']  - data_bb['valor_l1']

data_bb = data_bb.dropna()

data_bb.reset_index(drop = True, inplace = True)

treino_bb, teste_bb = split_treino_test(data_bb)
X_treino_bb = treino_bb.copy().drop(columns = ['target', 'datetime'])
X_teste_bb = teste_bb.copy().drop(columns = ['target', 'datetime'])

y_treino_bb = treino_bb[[ 'target']]
y_teste_bb = teste_bb[['target']]

In [11]:
# Fitando uma regressão logística
model = keras.Sequential()
model.add(keras.layers.Dense(1, activation='sigmoid', input_dim=7))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.fit(
  x=X_treino_bb,
  y=y_treino_bb,
  epochs=5,
  batch_size=64, validation_data=(X_teste_bb, y_teste_bb)
)

Epoch 1/5
1329/1329 [==============================] - 4s 1ms/step - loss: 1444.2661 - binary_accuracy: 0.5121 - val_loss: 17.8950 - val_binary_accuracy: 0.5617
Epoch 2/5
1329/1329 [==============================] - 2s 1ms/step - loss: 15.9393 - binary_accuracy: 0.5130 - val_loss: 41.4869 - val_binary_accuracy: 0.4371
Epoch 3/5
1329/1329 [==============================] - 2s 1ms/step - loss: 18.4396 - binary_accuracy: 0.5125 - val_loss: 20.8928 - val_binary_accuracy: 0.4378
Epoch 4/5
1329/1329 [==============================] - 2s 1ms/step - loss: 16.3369 - binary_accuracy: 0.5141 - val_loss: 13.0936 - val_binary_accuracy: 0.5598
Epoch 5/5
1329/1329 [==============================] - 2s 1ms/step - loss: 17.5704 - binary_accuracy: 0.5108 - val_loss: 7.1643 - val_binary_accuracy: 0.5166


In [12]:
score = model.evaluate(X_teste_bb, y_teste_bb, verbose=0)
score

[7.164344787597656, 0.5166071653366089]